In [1]:
# Import necessary libraries
import os
import cv2
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
import albumentations as A

# Load image and mask paths
data_dir = r'C:\Users\USER\Desktop\5Ctech\Data'
image_paths = glob(os.path.join(data_dir, '**', '*.tif'), recursive=True)
mask_paths = [img for img in image_paths if '_mask' in img]
image_paths = [img for img in image_paths if '_mask' not in img]

# Sort images and masks to align them properly
image_paths.sort()
mask_paths.sort()

# CLAHE preprocessing function
def clahe_preprocess(image):
    img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl_img = clahe.apply(img)
    norm_img = cl_img / 255.0  # Normalize the image
    return norm_img

# Load and preprocess images and masks
images = [clahe_preprocess(img) for img in image_paths]
masks = [cv2.imread(mask, cv2.IMREAD_GRAYSCALE) / 255.0 for mask in mask_paths]  # Normalize masks

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)

# Augmentation using albumentations
augmentation = A.Compose([
    A.HorizontalFlip(),
    A.VerticalFlip(),
    A.RandomRotate90(),
])

def augment_image(image, mask):
    augmented = augmentation(image=image, mask=mask)
    return augmented['image'], augmented['mask']

# Apply augmentation on the training set
X_train_aug = []
y_train_aug = []
for img, mask in zip(X_train, y_train):
    aug_img, aug_mask = augment_image(img, mask)
    X_train_aug.append(aug_img)
    y_train_aug.append(aug_mask)

# Optionally save the preprocessed and augmented data
# np.save('X_train.npy', X_train_aug)
# np.save('y_train.npy', y_train_aug)
